In [1]:
import numpy as np
from scipy.optimize import fsolve
import matplotlib.pyplot as plt
%matplotlib inline

Dario's results are in an email on 06/02/2020: 

I have gone back to those preliminary results for SiO2 (quite a bit of effort to remind myself of what was done!).

It would seem that SiO2 may have a small preference of going into the silicate, but the numbers are small and the noise is still large:

mu_SiO2(core) = 0 +- 0.3
mu_SiO2(silicate) = -0.3 +- 0.2

This is at 124.4 GPa and 5500 K.

The main difficulty with statistics is that we effectively lose a factor of 2 compared with FeO, as two oxygen atoms only make one molecule instead of two.
But we can lean on it and try to improve it, I’ll let you know how we get on.

To get the form of $K_d$ we follow the PRX paper using the same notation. The chemical potentials for SiO$_2$ in the core and magma ocean are

$$ \mu_{SiO_2}^c = \mu_{Si}^c + 2\mu_O^c $$ 
$$ \mu_{SiO_2}^m = \mu_{Si}^m + 2\mu_O^m $$ 

which are equal at equilibrium as given by (25). Separating the configurational parts we get

$$k_B T \ln c_{Si}^c + 2k_B T \ln c_{O}^c + \tilde{\mu}_{SiO_2}^c = k_B T \ln c_{Si}^m + 2k_B T \ln c_{O}^m + \tilde{\mu}_{SiO_2}^m$$ 

or

$$k_B T \ln \left[ \frac{c_{Si}^c}{c_{Si}^m} \right] + 2k_B T \ln \left[ \frac{c_{O}^c} {c_O^m} \right] = - \left( \tilde{\mu}_{SiO_2}^c  - \tilde{\mu}_{SiO_2}^m \right) $$ 

$$k_B T \ln \frac{c_{Si}^c}{c_{Si}^m} \left[ \frac{c_{O}^c} {c_O^m} \right]^2 = - \left( \tilde{\mu}_{SiO_2}^c  - \tilde{\mu}_{SiO_2}^m \right) $$ 

$$\frac{c_{Si}^c}{c_{Si}^m} \left[ \frac{c_{O}^c} {c_O^m} \right]^2 = \exp{ - \left( \frac{(\tilde{\mu}_{SiO_2}^c  - \tilde{\mu}_{SiO_2}^m)}{k_B T} \right)}$$ 

In the literature $K_d$ is usually written

$$ {\rm SiO_2^m \Longleftrightarrow Si^c + 2O^c} \Rightarrow K_d^s = \frac{c_{Si}^c (c_O^c)^2}{c_{SiO_2}^m} $$

$$ {\rm SiO_2^m + 2Fe^c \Longleftrightarrow 2FeO^m + Si^c} \Rightarrow K_d^e = \frac{(c_{FeO}^m)^2}{(c_{Fe}^c)^2} \frac{c_{Si}^c}{c_{SiO2}^m} $$

Therefore we obtain

$$ K_d^d = (c_O^m)^2 \exp{ - \left( \frac{(\tilde{\mu}_{SiO_2}^c  - \tilde{\mu}_{SiO_2}^m)}{k_B T} \right)} $$

$$ K_d^e = \left[ \frac{c_{O}^m} {c_O^c} \right]^2 \left[ \frac{c_{Fe}^m} {c_{Fe}^c} \right]^2 \exp{ - \left( \frac{(\tilde{\mu}_{SiO_2}^c  - \tilde{\mu}_{SiO_2}^m)}{k_B T} \right)} $$

Note here that I am assuming that Fe existing in the MO as FeO and Si exists as SiO$_2$. 

In [12]:
kb   = 8.617e-5
T    = 5500.0

mu_SiO2_c     =  0 
mu_SiO2_c_max =  0.3
mu_SiO2_c_min = -0.3

mu_SiO2_s     = -0.3
mu_SiO2_s_max = -0.1 
mu_SiO2_s_min = -0.5

print(mu_SiO2_c_min     - mu_SiO2_s_min)

Kd_max = np.exp(-(mu_SiO2_c_max - mu_SiO2_s_min)/(kb*T))
Kd_min = np.exp(-(mu_SiO2_c_min - mu_SiO2_s_max)/(kb*T))
Kd_ave = np.exp(-(mu_SiO2_c     - mu_SiO2_s    )/(kb*T))

print("exp(dmu/kT)...")
print("D min = ", Kd_max)
print("D max = ", Kd_min)
print("D     = ", Kd_ave)

0.2
exp(dmu/kT)
D min =  0.18488982178945249
D max =  1.525006690561857
D     =  0.5309973778143395


In [22]:
# Rough compositions using Dario's compositions in PRX paper:
# Liquid with 148 Fe atoms and 7 O atoms 
# Silicate melt with 28 Mg atoms, 4 Fe atoms, 32 Si atoms, and 96 O atoms

cOc  = 7  /(148+7)
cFec = 148/(148+7)

cSim = (32+64) /(95+32+28+3)
cFem = (4 +4)  /(95+32+28+3)
cMgm = (28+28) /(95+32+28+3)
cOm  = 96      /(95+32+28+3)

print('Fe   = ', cFec)
print('O    = ', cOc)
print('SiO2 = ', cSim)
print('FeO  = ', cFem)
print('MgO  = ', cMgm)
print('O MO = ', cOm)

95
Fe   =  0.9548387096774194
O    =  0.04516129032258064
SiO2 =  0.6075949367088608
FeO  =  0.05063291139240506
MgO  =  0.35443037974683544
O MO =  0.6075949367088608


In [25]:
Kdd_max = Kd_max * cOm**2
Kdd_min = Kd_min * cOm**2
Kdd_ave = Kd_ave * cOm**2

print("Kdd...")
print("min = ", Kdd_max)
print("max = ", Kdd_min)
print("    = ", Kdd_ave)

Kde_max = Kd_max * (cOm/cOc)**2 * (cFem/cFec)**2
Kde_min = Kd_min * (cOm/cOc)**2 * (cFem/cFec)**2
Kde_ave = Kd_ave * (cOm/cOc)**2 * (cFem/cFec)**2

print("Kde...")
print("min = ", Kde_max)
print("max = ", Kde_min)
print("    = ", Kde_ave)

Kdd...
min =  0.06825607264907843
max =  0.5629891708146961
    =  0.1960291553411694
Kde...
min =  0.09410536379665915
max =  0.7761990790985213
    =  0.27026745404727914
